# Assignment no 4
Name: Vishal Pattar \
Roll no: 33557 \
Class: TE AIML \
Subject: Artificial Intelligence for Cyber Security

In [145]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [146]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [147]:
df = pd.read_csv('./titanic.csv')
df.shape

(891, 12)

In [148]:
df.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
276,277,0,3,"Lindblom, Miss. Augusta Charlotta",female,45.0,0,0,347073,7.750,NaN,S
236,237,0,2,"Hold, Mr. Stephen",male,44.0,1,0,26707,26.000,NaN,S
389,390,1,2,"Lehmann, Miss. Bertha",female,17.0,0,0,SC 1748,12.000,NaN,C
353,354,0,3,"Arnold-Franchi, Mr. Josef",male,25.0,1,0,349237,17.800,NaN,S
279,280,1,3,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0,1,1,C.A. 2673,20.250,NaN,S
311,312,1,1,"Ryerson, Miss. Emily Borie",female,18.0,2,2,PC 17608,262.375,B57 B59 B63 B66,C
619,620,0,2,"Gavey, Mr. Lawrence",male,26.0,0,0,31028,10.500,NaN,S
16,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.125,NaN,Q
716,717,1,1,"Endres, Miss. Caroline Louise",female,38.0,0,0,PC 17757,227.525,C45,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S


In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [150]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [151]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [152]:
for x in df.columns:
    print([x,df[x].nunique()])

['PassengerId', 891]
['Survived', 2]
['Pclass', 3]
['Name', 891]
['Sex', 2]
['Age', 88]
['SibSp', 7]
['Parch', 7]
['Ticket', 681]
['Fare', 248]
['Cabin', 147]
['Embarked', 3]


In [153]:
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
def search_substring(big_string, substring_list):
    for substring in substring_list:
        if substring in big_string:
            return substring
    return substring_list[-1]

In [154]:
def get_title(string):
    import re
    regex = re.compile(r'Mr|Don|Major|Capt|Jonkheer|Rev|Col|Dr|Mrs|Countess|Dona|Mme|Ms|Miss|Mlle|Master', re.IGNORECASE)
    results = regex.search(string)
    if results != None:
        return(results.group().lower())
    else:
        return(str(np.nan))

In [155]:
title_dictionary = {
    "capt":"Officer", 
    "col":"Officer", 
    "major":"Officer", 
    "dr":"Officer",
    "jonkheer":"Royalty",
    "rev":"Officer",
    "countess":"Royalty",
    "dona":"Royalty",
    "lady":"Royalty",
    "don":"Royalty",
    "mr":"Mr",
    "mme":"Mrs",
    "ms":"Mrs",
    "mrs":"Mrs",
    "miss":"Miss",
    "mlle":"Miss",
    "master":"Master",
    "nan":"Mr"
}

In [156]:
df['Deck'] = df['Cabin'].map(lambda x: search_substring(str(x), cabin_list))
df['Title'] = df['Name'].apply(get_title)

In [157]:
df = df.drop(columns=['PassengerId','Name','Ticket','Cabin'], axis=1)

In [158]:
df_obj = df.select_dtypes(include=['object'])
df_num = df.select_dtypes(exclude=['object'])

In [159]:
num_imputer = SimpleImputer(strategy="median")
df["Age"] = num_imputer.fit_transform(df[["Age"]])
df["Fare"] = num_imputer.fit_transform(df[["Fare"]])

In [160]:
cat_imputer = SimpleImputer(strategy="most_frequent")
imputed_embarked = cat_imputer.fit_transform(df[["Embarked"]])
df["Embarked"] = pd.Series(imputed_embarked.flatten(), name="Embarked")

In [161]:
label_encoder = LabelEncoder()
for c in df_obj.columns:
    df[c] = label_encoder.fit_transform(df[c])

In [162]:
scaler = StandardScaler()
df[["Age", "Fare"]] = scaler.fit_transform(df[["Age", "Fare"]])

In [163]:
X = df.drop("Survived", axis=1)
y = df["Survived"]

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [164]:
df.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Title
83,0,1,1,-0.104637,0,0,0.299922,2,8,11
725,0,3,1,-0.719436,0,0,-0.474005,2,8,11
624,0,3,1,-0.642586,0,0,-0.324253,2,8,11
112,0,3,1,-0.565736,0,0,-0.486337,2,8,11
695,0,2,1,1.739759,0,0,-0.376603,2,8,11
